// can't yet format YamlFrontmatter (["title: Caffeine and endurance"; "category: Datasets"; "categoryindex: 1"; "index: 3"], Some { StartLine = 2 StartColumn = 0 EndLine = 6 EndColumn = 8 }) to pynb markdown

[![Binder](/datasets/img/badge-binder.svg)](https://mybinder.org/v2/gh/plotly/Plotly.NET/gh-pages?filepath=03_CaffeineAndEndurance.ipynb)&emsp;
[![Script](/datasets/img/badge-script.svg)](/datasets/03_CaffeineAndEndurance.fsx)&emsp;
[![Notebook](/datasets/img/badge-notebook.svg)](/datasets/03_CaffeineAndEndurance.ipynb)

# The _caffeine and endurance_ dataset

**Table of contents**

- [Description]()
- [How to use]()
- [Examples]()

## Description

Endurance times for 9 well-trained cyclists, on each of 4 doses of caffeine (0, 5, 9, 13 mg) with 1 line per subject.
Taken from [Lawrence H. Winner, University of Florida](http://archived.stat.ufl.edu/personnel/usrpages/winner.shtml):
- [Data](http://users.stat.ufl.edu/~winner/data/caffeine1.dat)
- [Description](http://users.stat.ufl.edu/~winner/data/caffeine1.txt)

Original literature: W.J. Pasman, M.A. van Baak, A.E. Jeukendrup, A. de Haan (1995). _The Effect of Different Dosages of Caffeine on Endurance Performance Time_, International Journal of Sports Medicine, Vol. 16, pp225-230.


## How to use




In [1]:
#r "nuget: FSharp.Data"
#r "nuget: Deedle"

open FSharp.Data
open Deedle
open System.Text.RegularExpressions

let rawDataCaffeine = Http.RequestString @"https://raw.githubusercontent.com/fslaborg/datasets/main/data/CaffeineAndEndurance(wide).dat"
// This data format features a char column-wise structure. To transform it into a seperator-delimited format, we have to replace the multiple spaces via Regex:
let regexCaffeine = [Regex("[ ]{2,}1"), "1"; Regex("[ ]{2,}"), "\t"; Regex("\n\t"), "\n"]
let rawDataCaffeineAdapted = 
    regexCaffeine
    |> List.fold (fun acc (reg,rep) -> reg.Replace(acc, rep)) rawDataCaffeine

let dataFrame = Frame.ReadCsvString(rawDataAdapted, hasHeaders = false, separators = "\t", schema = "Subject ID, no Dose, 5 mg, 9 mg, 13 mg")

// Otherwise, the following already adapted dataset can be used:
let rawData2 = Http.RequestString @"https://raw.githubusercontent.com/fslaborg/datasets/main/data/CaffeineAndEndurance(wide)_adapted.tsv"

let dataFrame2 = Frame.ReadCsvString(rawData2, hasHeaders = false, separators = "\t", schema = "Subject ID, no Dose, 5 mg, 9 mg, 13 mg")

dataFrame2.Print()


## Examples

This example is taken from the FsLab datascience tutorial [t-test]()
(WIP)




In [2]:
#r "nuget: FSharp.Stats, 0.4.2"
#r "nuget: Plotly.NET, 2.0.0-preview6"

open FSharp.Stats
open FSharp.Stats.Testing
open Plotly.NET

// We want to compare the subjects' performances under the influence of 13 mg caffeine and in the control situation.
let dataCaffeineNoDose, dataCaffeine13mg =
    let getVectorFromCol col = 
        dataCaffeineAsFrame
        |> Frame.getCol col
        |> Series.values
        |> vector
    getVectorFromCol "no Dose", getVectorFromCol "13 mg"

// 
let visualizePairedData = 
    Seq.zip dataCaffeineNoDose dataCaffeine13mg
    |> Seq.mapi (fun i (control,treatment) -> 
        let participant = "Person " + string i 
        Chart.Line(["no dose", control; "13 mg", treatment], Name = participant)
        )
    |> Chart.Combine
    |> Chart.withX_AxisStyle ""
    |> Chart.withY_AxisStyle("endurance performance", MinMax = (0.,100.))


In [ ]:
visualizePairedData


No value returned by any evaluator

The function for pairwise t tests can be found at `FSharp.Stats.Testing.TTest.twoSamplePaired`. Note, that the order of the elements in each vector must be the same, so a pairwise comparison
can be performed.





In [4]:
let twoSamplePairedResult = TTest.twoSamplePaired dataCaffeineNoDose dataCaffeine13mg


No value returned by any evaluator